Chapter 6 Stacks and Queues<br><br>

6.1 Abstract Data Types<br><br>

An ADT is not a data structure, but it does tell us about data structures.<br>
An ADT answers two main questions:
1. What is the data to be stored or represented?
2. What can we do with the data?

When we give an ADT, we will list the names of the methods that will be present, what kind of input they take, and what is their expected output.<br>
The ADT may also describe error situations and what should happen if they occur.<br>
A data structure is an implementation of an ADT.
The ADT tells us what methods the data structure will implement. However, the ADT does not give an hints or prescriptions for how the data structure is implemented.<br>
The ADT should be independent of all concerns about its implementation.

6.2 The Stack ADT<br><br>

- push - add a new item to the stack.
- pop - remove and return the next item in Last In First Out (LIFO) ordering.
- peek - return the next item in LIFO ordering.
- size - returns the number of items in the stack.
- isempty - returns True if the stack has no elements False otherwise.

This ADT can be implemented quite easily using a list. We will implement it with a class called ListStack.

In [1]:
class ListStack:
    def __init__(self):
        self._L = []

    def push(self, item):
        self._L.append(item)
    
    def pop(self):
        return self._L.pop()

    def peek(self):
        return self._L[-1]
    
    def __len__(self):
        return len(self._L)
    
    def isempty(self):
        return len(self._L) == 0

In [2]:
Lis = ListStack()
Lis.push(1)
Lis.push(2)
Lis.push(3)
print(Lis.peek())
print(Lis.pop())
print(len(Lis))
print(Lis.isempty())

3
3
2
False


The Stack class above illustrates the object-oriented strategy of composition (the Stack has a list).<br>
The Python builtin list is doing all the heavy lifting.

In [3]:
from ds2.stack import ListStack

class BadStack(ListStack):
    def push(self, item):
        self._L.insert(0, item)

    def pop(self):
        return self._L.pop(0)
    
    def peek(self):
        return self._L[0]

A simple asymptotic analysis shows why this implementation is far less efficient. Inserting a new item into a list requires that all the other items in the list have to move over to make room for the new item. If we insert at the beginning of the list, then every item has to be copied to a new position. Thus, the insert call in push takes O(n) time. Similarly, if we pop an item at the beginning of the list, then every other item in the list gets moved over one space to fill in the gap. Thus, the list.pop call in our pop method will take O(n) time as well.

6.3 The Queue ADT<br><br>

- enqueue(item) - add a new item to the queue.
- dequeue() - remove and return the next item in the First In First Out(FIFO) ordering.
- peek() - return the next item in the queue in FIFO order.
- __ len __() - return the number of items in the queue.
- isempty() - return True if the queue has no items, False otherwise.

In [4]:
class ListQueueSimple:
    def __init__(self):
        self._L = []

    def enqueue(self, item):
        self._L.append(item)

    def dequeue(self):
        return self._L.pop(0)
    
    def peek(self):
        return self._L[0]
    
    def __len__(self):
        return len(self._L)
    
    def isempty(self):
        return len(self._L) == 0

Calling pop(0) was a bad thing to do. Yes, it takes time proportional to the length of the list. If we dequeue off the end of the list, we would have to enqueue by inserting into the front of the list. That’s bad too.<br><br>

Here’s a different idea. Let’s not really delete things from the front of the list. Instead, we’ll ignore them by keeping the index of the head of the queue.

In [18]:
class ListQueueFakeDelete:
    def __init__(self):
        self._head = 0
        self._L = []

    def enqueue(self, item):
        self._L.append(item)

    def peek(self):
        return self._L[self._head]
    
    def dequeue(self):
        item = self.peek()
        self._head += 1
        return item
    
    def __len__(self):
        return len(self._L) - self._head
    
    def isempty(self):
        return len(self) == 0 

In [19]:
L=ListQueueFakeDelete()
L.enqueue(1)
L.dequeue()
L.isempty()

True

This implementation never gets rid of old items after they have been dequeued. Even if it deleted them, it still keeps a place in the list for them. This is a kind of lazy update.

In [20]:
class ListQueue(ListQueueFakeDelete):
    def dequeue(self):
        item = self._L[self._head]
        self._head += 1
        if self._head > len(self._L)//2:
            self._L = self._L[self._head:]
            self._head = 0
        return item

So, ”on average”, the cost per item is constant. This kind of lazy update is very important. Technically, pop() can also take linear time for some calls but on average, the cost is constant per operation. The same idea makes append fast.

6.4 Dealing with errors<br><br>

In the case of a stack, it is never correct usage to pop from an empty stack. Thus, it makes sense that someone using our Stack class should have their program crash and see an error message if they attempt to call pop when there are no items left on the stack.

In [21]:
s = ListStack()
s.push(5)
s.pop()
s.pop()

IndexError: pop from empty list

If we look at the error message, it seems pretty good. It says we tried to pop from empty list. But if you look at the code, you might ask, ”What list?” We could catch the exception in our pop method and raise a different error so the source of the problem is more obviously the user’s code. Then, a user, might have to try to understand our class in order to backtrack to understand what they did wrong in their code. Instead, give them an error that explains exactly what happened.

In [22]:
from ds2.stack import ListStack

class AnotherStack(ListStack):
    def pop(self):
        try:
            return self._L.pop()
        
        except IndexError:
            raise RuntimeError("pop from empty stack")
        
s = AnotherStack()
s.push(5)
s.pop()
s.pop()

RuntimeError: pop from empty stack

This gives clarity to the user where he/she might have went wrong.